# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [33]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [2]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [3]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': 'eef767a4-2397-4799-b783-7afd524b722c',
 'response_time': 1.13,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.7304634,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': '1. 더 마고 그릴 신세계강남 · 2. 모던눌랑 강남고속터미널 · 3. 에토레 파미에스테이션 '
                         '· 4. 자주테이블 신세계강남 · 5. 쓰리버즈 센트럴시티점 · 6. 더 키친 일뽀르노 신세계',
              'raw_content': None,
              'score': 0.6907474,
              'title': "'강남신세계백화점 레스토랑' 맛집 빅데이터 추천순위 Top10",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8

In [4]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

"강남 신세계백화점에서 점심 식사로 추천할 만한 곳으로는 '101번지남산돈까스'와 '한우리 센트럴시티점'이 있습니다. 돈까스와 한우국수전골 맛집으로 평이 좋고, 캐주얼하게 즐기기 좋아요. 또한, 신세계백화점 내 다양한 식당과 맛집들도 있으니 쇼핑과 함께 즐기기 좋습니다."

In [49]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계에서 점심 먹기 좋은 곳으로는 다양한 레스토랑이 있는데, 대표적으로 한식, 일식, 양식 등을 즐길 수 있는 곳들이 있습니다. 특히, 신세계백화점 내에 위치한 레스토랑들과 주변 인기 맛집들을 고려하면, 깔끔하고 다양한 메뉴를 제공하는 곳을 선택하시면 좋을 것 같아요. 만약 구체적인 추천이 필요하시면 더 알려드릴게요!'

In [38]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose=True)



In [51]:
agent_executor.invoke({'input': '나는 승은이야'})



> Entering new AgentExecutor chain...
반가워, 승은이님! 오늘 무엇을 도와드릴까요?

> Finished chain.


{'input': '나는 승은이야',
 'chat_history': [HumanMessage(content='나는 유태영이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워, 유태영님! 오늘 어떤 도움을 드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='나는 승은이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워, 승은님! 어떤 도움이나 궁금한 점이 있으시면 말씀해 주세요.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='나는 승은이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워, 승은이님! 오늘 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})],
 'output': '반가워, 승은이님! 오늘 무엇을 도와드릴까요?'}